In [1]:
import tkp.db  # NOTE: this only works with the queryqueue branch of TraP
import logging
from tkp.db.model import Dataset, Assocxtrsource, Extractedsource, Runningcatalog, Image
from sqlalchemy.orm import aliased
from sqlalchemy.sql.functions import max
from sqlalchemy.sql.expression import select

logging.basicConfig()

### Database connection settings

In [2]:
engine = 'postgresql'
host = 'localhost'
port = 5433
user = 'gijs'
password = user
database = user
query_loglevel = logging.INFO  # Set to INFO to see queries, otherwise WARNING

### connect to the database

In [3]:
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)
db = tkp.db.Database(engine=engine, host=host, port=port
                     user=user, password=password, database=database)
db.connect()
session = db.Session()

INFO:sqlalchemy.engine.base.Engine:select version()
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:select current_schema()
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:show standard_conforming_strings
INFO:sqlalchemy.engine.base.Engine:{}


### Execute the query

In [4]:
# maximum timestamps per runcat and band
a_1 = aliased(Assocxtrsource, name='a_1')
a_2 = aliased(Assocxtrsource, name='a_2')
e_1 = aliased(Extractedsource, name='e_1')
last_ts_per_band = session.query(max(a_2.id).label('assoc_id')).\
        join(r_1, a_2.runcat==r_1.id).\
        join(e_2, a_2.xtrsrc==e_2.id).\
        join(i_2, e_2.image==i_2.id).\
        group_by(r_1.id, i_2.id).\
        subquery(name='last_ts_per_band')

        
        
# Select peak flux per runcat at last timestep (over all bands) 
a_1 = aliased(Assocxtrsource, name='a_1')
a_2 = aliased(Assocxtrsource, name='a_2')
e_1 = aliased(Extractedsource, name='e_1')
last_ts_fmax = session.query(last_ts_per_band).\
        join(a_1, a_1.id==last_ts_per_band.c.assoc_id).\
        join(e_1,  a_1.xtrsrc==e_1.id).\
        group_by(a_1.runcat).\
        subquery(name='last_ts_fmax')

        
match_assoc = aliased(Assocxtrsource, name='match_assoc')
match_ex = aliased(Extractedsource, name='match_ex')
r = aliased(Runningcatalog, name='r')
match_img = aliased(Image, name='match_img')
query = session.query(last_ts_fmax).\
        join(match_assoc, match_assoc.runcat==last_ts_fmax.c.runcat_id).\
        join(match_ex,
             match_assoc.xtrsrc==match_ex.id and 
             match_ex.f_int==last_ts_fmax.max_flux).\
        join(r, r.id==last_ts_fmax.runcat_id).\
        join(match_img, match_ex.image==match_img.id).\
        subquery()

print(query)

NameError: name 'r_1' is not defined